In [2]:
li=["pelvis_x","pelvis_y","pelvis_z",
    "spine1_x","spine1_y","spine1_z",
    "spine2_x","spine2_y","spine2_z",
    "spine3_x","spine3_y","spine3_z",
    "neck_x","neck_y","neck_z",
    "head_x","head_y","head_z",
    "left_clavicle_x","left_clavicle_y","left_clavicle_z",
    "right_clavicle_x","right_clavicle_y","right_clavicle_z",
    "left_shoulder_x","left_shoulder_y","left_shoulder_z",
    "right_shoulder_x","right_shoulder_y","right_shoulder_z",
    "left_elbow_x","left_elbow_y","left_elbow_z",
    "right_elbow_x","right_elbow_y","right_elbow_z",
    "left_wrist_x","left_wrist_y","left_wrist_z",
    "right_wrist_x","right_wrist_y","right_wrist_z",
    "left_hip_x","left_hip_y","left_hip_z",
    "right_hip_x","right_hip_y","right_hip_z",
    "left_knee_x","left_knee_y","left_knee_z",
    "right_knee_x","right_knee_y","right_knee_z",
    "left_ankle_x","left_ankle_y","left_ankle_z",
    "right_ankle_x","right_ankle_y","right_ankle_z",
    "left_foot_x","left_foot_y","left_foot_z",
    "right_foot_x","right_foot_y","right_foot_z"]
len(li)/3

22.0

In [50]:
import os
import json
import random
import numpy as np
from scipy.stats import bernoulli


In [108]:
SEED = 0
random.seed(SEED)
np.random.random(SEED)
rng_split = np.random.RandomState(SEED)
rng_score = np.random.RandomState(SEED + 1)

# coordinate indices
indices = [15,16, # head
           12,13, # neck
           27,28, # right shoulder
           33,34, # right elbow
           39,40, # right wrist
           24,25, # left shoulder
           30,31, # left elbow
           36,37, # left wrist
           45,46, # right hip
           51,52, # right knee
           57,58, # right anckle
           42,43, # left hip
           48,49, # left knee
           54,55, # left anckle
           15,16, # head
           15,16, # head
           15,16, # head
           15,16] # head
# test vs train
te = bernoulli(0.2)

fd_name = [name for name in os.listdir(r'C:\Users\user\Desktop\연구\7. CATGCN\AI_DataSet_MP4_JSON') if name != '.DS_Store']
pose_id = [name.split('_')[0] for name in fd_name if name != '.DS_Store']


In [53]:
fd_name[0]

'1_일반 걷기'

In [126]:
base_path = r'C:\Users\user\Desktop\연구\7. CATGCN\AI_DataSet_MP4_JSON'

for i in range(0,len(fd_name)):
    iact = int(pose_id[i])
    ifd = fd_name[i]

    folder = os.path.join(base_path, ifd)
    fname = sorted([x for x in os.listdir(folder) if x.lower().endswith('.json')])

    for j, ifn in enumerate(fname):
        file_path = os.path.join(folder, ifn)   
        ### Opening JSON file
        with open(file_path, 'r', encoding='utf-8') as openfile:
            # Reading from json file
            json_object = json.load(openfile)
            # print(json_object)
            
            # pose
            keypoints = json_object['annotations']['keypoints']
            nframe = len(keypoints)
            
            dta = []
            
            if nframe < 300:
                for iframe in range(0, nframe):
                    pose = list(np.array(keypoints[iframe])[indices])
                    pose[0:len(pose):2] = list(np.array(pose[0:len(pose):2])/1920)
                    pose[1:len(pose):2] = list(np.array(pose[1:len(pose):2])/1080)
                
                    pose = [round(p, 3) for p in pose]
                    
                    # score
                    score = []
                    for k in range(0,18):
                        score.append(round(float(0.7 + rng_score.rand()*0.2),3))

                    # Make a dictionary object
                    skt = list()
                    skt.append({"pose": pose, "score": score})
                    dta.append({"frame_index": iframe, "skeleton": skt})
                    
                for iframe in range(nframe, 300):
                    # Make a dictionary object
                    skt = list()
                    #skt.append({"pose": [], "score": []})
                    dta.append({"frame_index": iframe, "skeleton": skt})
                # print(dta)

            elif nframe >= 300:
                sampled = np.linspace(0, nframe-1, 300)
                sampled = np.round(sampled).astype(int)
                for ith_frame, iframe in enumerate(sampled):
                    pose = list(np.array(keypoints[iframe])[indices])
                    pose[0:len(pose):2] = list(np.array(pose[0:len(pose):2])/1920)
                    pose[1:len(pose):2] = list(np.array(pose[1:len(pose):2])/1080)
                    
                    pose = [round(p, 3) for p in pose]

                    # score
                    score = []
                    for k in range(0,18):
                        score.append(round(float(0.7 + rng_score.rand()*0.2),3))

                    # Make a dictionary object
                    skt = list()
                    skt.append({"pose": pose, "score": score})
                    dta.append({"frame_index": ith_frame, "skeleton": skt})
                # print(dta)
            ### Writing JSON file
            out = {"data": dta,
                    "label": "a"+str(iact),
                    "label_index": iact}  
        
            tmpname = ifn.split('_')

            if te.rvs(1)[0] == 1:
                outfname = base_path + '/data/fire_val/__a'+str(iact)+'_'+tmpname[1]+'_'+tmpname[2]+'_'+'f'+str(j)+'.json'
            else:
                outfname = base_path + '/data/fire_train/__a'+str(iact)+'_'+tmpname[1]+'_'+tmpname[2]+'_'+'f'+str(j)+'.json'
                
            with open(outfname, "w", encoding="UTF-8") as outfile:
                json.dump(out, outfile)
       
# label
te_name = [name for name in os.listdir(base_path + '/data/fire_val') if name != '.DS_Store']
val_label = {}
for i in range(0,len(te_name)):
    itr = te_name[i]
    iact = int((itr.split('_a')[1]).split('_')[0])
    print(iact)
    val_label[itr.split('.')[0]]= {
              "has_skeleton": True,
              "label": "a"+str(iact),
              "label_index": iact}

tr_name = [name for name in os.listdir(base_path + '/data/fire_train') if name != '.DS_Store']
train_label = {}
for i in range(0,len(tr_name)):
    itr = tr_name[i]
    iact = int((itr.split('_a')[1]).split('_')[0])
    print(iact)
    train_label[itr.split('.')[0]]= {
                "has_skeleton": True,
                "label": "a"+str(iact),
                "label_index": iact}
            
with open('./data/fire_val_label.json','a', encoding="UTF-8") as outfile2:
  json.dump(val_label, outfile2, indent = 4)

with open('./data/fire_train_label.json','a', encoding="UTF-8") as outfile3:
  json.dump(train_label, outfile3, indent = 4)



157
17
1
1
1
1
157
17
17
1
1
1
1
1
1
221


FileNotFoundError: [Errno 2] No such file or directory: './data/fire_val_label.json'

In [74]:
keypoints = json_object["annotations"]["keypoints"]

print("nframe:", len(keypoints))
print("type(keypoints[0]):", type(keypoints[0]))
print("len(keypoints[0]):", len(keypoints[0]))
print("type(keypoints[0][0]):", type(keypoints[0][0]))

# 첫 프레임 coords 길이로 관절수 추정
L = len(keypoints[0])
print("coords length:", L, "  joints(if xyz):", L/3)

print("indices length:", len(indices))
print("first 10 indices:", indices[:10])

# indices가 범위 안인지
print("max index:", max(indices), "within coords length?", max(indices) < L)

names = json_object["categories"]["keypoints"]
for idx in indices:
    print(idx, names[idx])


nframe: 273
type(keypoints[0]): <class 'list'>
len(keypoints[0]): 66
type(keypoints[0][0]): <class 'float'>
coords length: 66   joints(if xyz): 22.0
indices length: 36
first 10 indices: [15, 16, 12, 13, 27, 28, 33, 34, 39, 40]
max index: 58 within coords length? True
15 head_x
16 head_y
12 neck_x
13 neck_y
27 right_shoulder_x
28 right_shoulder_y
33 right_elbow_x
34 right_elbow_y
39 right_wrist_x
40 right_wrist_y
24 left_shoulder_x
25 left_shoulder_y
30 left_elbow_x
31 left_elbow_y
36 left_wrist_x
37 left_wrist_y
45 right_hip_x
46 right_hip_y
51 right_knee_x
52 right_knee_y
57 right_ankle_x
58 right_ankle_y
42 left_hip_x
43 left_hip_y
48 left_knee_x
49 left_knee_y
54 left_ankle_x
55 left_ankle_y
15 head_x
16 head_y
15 head_x
16 head_y
15 head_x
16 head_y
15 head_x
16 head_y


In [43]:
ifn = fname[0]
if ifn.split('.')[-1] == 'json':
    # print(ifn) 
    
    ### Opening JSON file
    with open(r'C:\Users\user\Desktop\연구\7. CATGCN\FIRE_raw - 복사본\fire_data\AI_DataSet_MP4_JSON\\' + ifd + '/' + ifn, 'r', encoding='utf-8') as openfile:
        # Reading from json file
        json_object = json.load(openfile)
        print(json_object)

        # pose
        keypoints = json_object['annotations']['keypoints']
        print(keypoints)
        nframe = len(keypoints)
        print(nframe)

{'animations': {'id': 1, 'action_category': 'B-01-01', 'motion_name': '일반 걷기', 'createtime': '2022-09-19', 'file_size': 3490224, 'worker': 'park ga hyeon', 'license': 'blue', 'length': 9.03, 'fps': 30, 'total_frame': 273, 'loop_start': 0, 'loop_end': 273, 'format': 'fbx', 'task': '구조대원', 'position': '소방교', 'site': '3D융합상용화지원센터'}, 'categories': {'keypoints': ['pelvis_x', 'pelvis_y', 'pelvis_z', 'spine1_x', 'spine1_y', 'spine1_z', 'spine2_x', 'spine2_y', 'spine2_z', 'spine3_x', 'spine3_y', 'spine3_z', 'neck_x', 'neck_y', 'neck_z', 'head_x', 'head_y', 'head_z', 'left_clavicle_x', 'left_clavicle_y', 'left_clavicle_z', 'right_clavicle_x', 'right_clavicle_y', 'right_clavicle_z', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'le

In [ ]:
np.linspace(0, 5,4)

array([0.        , 1.66666667, 3.33333333, 5.        ])

In [27]:
keypoints = json_object['annotations']['keypoints']

print("nframe =", len(keypoints))
print("type(keypoints[0]) =", type(keypoints[0]))
print("len(keypoints[0]) =", len(keypoints[0]))

# keypoints[0]의 첫 원소 타입 확인
print("type(keypoints[0][0]) =", type(keypoints[0][0]))

# 만약 keypoints[0][0]가 list면 케이스2(사람 차원 존재)일 확률 높음
if isinstance(keypoints[0][0], (list, tuple)):
    print(">> seems like [frame][person][coords]")
    print("n_person_in_frame0 =", len(keypoints[0]))
    print("len(coords_of_person0) =", len(keypoints[0][0]))
else:
    print(">> seems like [frame][coords]")
    print("len(coords_of_frame0) =", len(keypoints[0]))


nframe = 227
type(keypoints[0]) = <class 'list'>
len(keypoints[0]) = 66
type(keypoints[0][0]) = <class 'float'>
>> seems like [frame][coords]
len(coords_of_frame0) = 66


In [ ]:


           ### Opening JSON file
           with open('./AI_DataSet_MP4_JSON/'+ifd+'/'+ifn, 'r') as openfile:
             # Reading from json file
             json_object = json.load(openfile)
             #print(json_object)
             
           # pose
           keypoints = json_object['annotations']['keypoints']
           nframe = len(keypoints)
           if nframe >= 300:
             dta = []
             ith_frame = 1
             for iframe in list(np.linspace(1,nframe,299,dtype='int')):
               pose = list(np.array(keypoints[iframe-1])[indices])
               
               pose[0:len(pose):2] = list(np.array(pose[0:len(pose):2])/1920)
               pose[1:len(pose):2] = list(np.array(pose[1:len(pose):2])/1080)
               
               for i in range(0,len(pose)):
                 pose[i] = round(pose[i],3)

               # score
               score = []
               for i in range(0,18):
                 score.append(round(float(0.7 + np.random.random(1)*0.2),3))

               # Make a dictionary object
               skt = list()
               skt.append({"pose": pose, "score": score})
               dta.append({"frame_index": ith_frame, "skeleton": skt})
               ith_frame += 1
           else:  
             dta = []
             for iframe in range(1,nframe+1):
               pose = list(np.array(keypoints[iframe-1])[indices])
               
               pose[0:len(pose):2] = list(np.array(pose[0:len(pose):2])/1920)
               pose[1:len(pose):2] = list(np.array(pose[1:len(pose):2])/1080)
               
               for i in range(0,len(pose)):
                 pose[i] = round(pose[i],3)

               # score
               score = []
               for i in range(0,18):
                 score.append(round(float(0.7 + np.random.random(1)*0.2),3))

               # Make a dictionary object
               skt = list()
               skt.append({"pose": pose, "score": score})
               dta.append({"frame_index": iframe, "skeleton": skt})
               
             for iframe in range(nframe,300):
               # Make a dictionary object
               skt = list()
               #skt.append({"pose": [], "score": []})
               dta.append({"frame_index": iframe, "skeleton": skt})
               
           ### Writing JSON file
           out = {"data": dta,
                  "label": "a"+str(iact),
                  "label_index": iact}  
    
           tmpname = ifn.split('_')
           if te.rvs(1)[0] == 1:
             outfname = './kinetics-skeleton/kinetics_val/__a'+str(iact)+'_'+tmpname[1]+'_'+tmpname[2]+'_'+'f'+str(j)+'.json'
           else:
             outfname = './kinetics-skeleton/kinetics_train/__a'+str(iact)+'_'+tmpname[1]+'_'+tmpname[2]+'_'+'f'+str(j)+'.json'
             
           with open(outfname, "w", encoding="UTF-8") as outfile:
             json.dump(out, outfile)
             
             
    
# label

te_name = [name for name in os.listdir('./kinetics-skeleton/kinetics_val') if name != '.DS_Store']
val_label = {}
for i in range(0,len(te_name)):
    itr = te_name[i]
    iact = int((itr.split('_a')[1]).split('_')[0])
    print(iact)
    val_label[itr.split('.')[0]]= {
              "has_skeleton": True,
              "label": "a"+str(iact),
              "label_index": iact}

tr_name = [name for name in os.listdir('./kinetics-skeleton/kinetics_train') if name != '.DS_Store']
train_label = {}
for i in range(0,len(tr_name)):
    itr = tr_name[i]
    iact = int((itr.split('_a')[1]).split('_')[0])
    print(iact)
    train_label[itr.split('.')[0]]= {
                "has_skeleton": True,
                "label": "a"+str(iact),
                "label_index": iact}
            
with open('./kinetics-skeleton/kinetics_val_label.json','a', encoding="UTF-8") as outfile2:
  json.dump(val_label, outfile2, indent = 4)

with open('./kinetics-skeleton/kinetics_train_label.json','a', encoding="UTF-8") as outfile3:
  json.dump(train_label, outfile3, indent = 4)

